# Cvičenie 11: Nastupujte, prosím!

Každý, kto cestuje lietadlom zažije pocit pri odletovej bráne, že rad na lietadlo sa vôbec nepohne a určite musí existovať rýchlejší spôsob nastupovania cestujúcich. Na tomto cvičení porovnáme rôzne možnosti nástupu pasažierov do lietadla a vyhodnotíme, ktorá z nich je najrýchlejšia pomocou simulácií. Pre jednoduchosť už budeme mať vytvorený model lietadla aj pohyb pasažierov, sústredíme sa teda na optimálny spôsob pustenia ľudí na lietadlo. Pre lepšie pochopenie úlohy si môžete pozrieť [video na kanáli CGP Grey](https://www.youtube.com/watch?v=oAHbLRjF0vo), ktoré bolo inšpiráciou cvičenia.

## Krok 1: Oboznámenie sa s kódom

[Stiahnite si predpripravené riešenie s niekoľkými už implementovanými triedami.](sources/lab11/lab11.zip) V riešení nájdete štyri súboru, z ktorých tri sú už plne implementované. Najprv sa oboznámte s kódom, pričom simulácia bude používať nasledujúce modely:

* `Passenger` (v súbore `passenger.py`) - trieda reprezentuje jedného cestujúceho s prideleným miestom a batožinou;
* `Plane` (v súbore `plane.py`) - trieda reprezentuje lietadlo s určenou dĺžkou. Lietadlo budeme reprezentovať ako zoznam zoznamov, napr. takto:

![](sources/lab11/plane.jpg)

kde stredný prázdny riadok je chodba a dva krajné stĺpce (označené X) sú iba pomocné stĺpce a zjednodušujú simuláciu. Na každej pozícii v tomto poli bude zoznam cestujúcich, ktorí sa nachádzajú na danej pozícii. **Pozor:** pri reprezentácii lietadla pomocou dvojrozmerného poľa sú v riadkoch sedadlá s rovnakým písmenom, rady sedadiel sú vo stĺpcoch!

* `Boarding` a jeho podtriedy (v súbore `boarding.py`) - prázdne implementácie tried reprezentujúcich spôsob nastupovania do lietadla. Spôsob bude definovaný poradím pridávania pasažierov na štartovaciu pozíciu v lietadle. Trieda `Boarding` definuje všeobecnú funkcionalitu, podtriedy špecifikujú už konkrétny spôsob nastupovania, resp. pridávania cestujúcich do lietadla.

### 1.1 `globals.py`

Súbor `globals.py` obsahuje iba dve pomocné premenné pre jednoduchý prechod medzi označeniami sedadiel pomocou písmen a indexom riadku v poli reprezentujúcom naše lietadlo. Konkrétne definujeme slovník `SEAT_TO_IDX`, ktorý prekonvertuje písmenové označenie na index riadku, a slovník `IDX_TO_SEAT` pre opačnú konverziu.

In [ ]:
SEAT_TO_IDX = {
    "A": 6,
    "B": 5,
    "C": 4,
    "D": 2,
    "E": 1,
    "F": 0
}

IDX_TO_SEAT = {
    0: "F",
    1: "E",
    2: "D",
    4: "C",
    5: "B",
    6: "A"
}

### 1.2 `Passenger`

Trieda `Passenger` definuje pasažiera a jeho správanie a to nasledovným spôsobom:

* konštruktor triedy má tri parametre: `row` (číslo radu - v poli to definuje stĺpec), `seat` (písmeno sedadla cestujúceho - v poli to definuje riadok) a `no_of_bags` (počet kusov batožiny). Po kontrole vstupov konštruktor definuje nasledovné vnútorné premenné:
  * `self.row` – číslo radu, kde cestujúci má miesto (parameter `row`)
  * `self.seat` – písmeno sedadla (parameter `seat`)
  * `self.bags` – vyjadruje počet krokov, ktorý cestujúci potrebuje na to, aby svoju batožinu dal do odkladacieho priestoru nad hlavou (parameter `no_of_bags * 4`) 
  * `self.plane` – smerník na lietadlo, v ktorom sa nachádza cestujúci; v konštruktore nastavený na `None`
  * `self.current_position` – vyjadruje aktuálnu polohu cestujúceho v lietadle, v konštruktore nastavená na `[None, None]`
* `get_position(self)` – metóda vracia pozíciu cestujúceho v poli (riadok a stĺpec v zozname zoznamov), ktoré reprezentuje lietadlo v ktorom sa cestujúci nachádza. Ak cestujúci ešte nebol pridaný do lietadla, metóda vracia `None`.
* `get_seat(self)` – metóda vracia pozíciu sedadla cestujúceho v lietadle (riadok a stĺpec v zozname zoznamov).
* `add_to_plane(self, plane)` – metóda pridá cestujúceho do lietadla. Metóda má jeden parameter: smerník na lietadlo, do ktorého chceme pridať cestujúceho. V metóde aktualizujeme hodnotu vnútornej premennej `plane` a dvojicu čísel `current_position` na `[0, 3]` (*0* reprezentuje nultý rad sedadiel, *3* je chodba – index stĺpca a riadku v dvojrozmernom poli reprezentujúcom lietadlo).
* `can_sit(self)` – metóda určí, či cestujúci má voľnú cestu ku svojmu miestu ak už stojí v danom rade. Ak cestujúci má miesto pri chodbe, metóda vracia vždy `True`, v opačnom prípade vráti `True` ak sedadlá medzi sedadlom cestujúceho a chodbou nie sú obsadené (napr.: ak si cestujúci chce sadnúť na 4E, ale niekto sedí na 4D, metóda vracia `False`). Funkcia vracia `False` ak cestujúci ešte nebol pridaný do lietadla, alebo nestojí na chodbe. Na kontrolu voľnosti pozícií použijeme metódu `is_empty` z triedy `Plane`.
* `forced_to_move(self, x, y)` – metóda reprezentuje vynútený pohyb cestujúceho (samotný cestujúci sa nechce pohnúť, ale je to nevyhnutné pretože prekáža niekomu inému). Metóda má dva parametre – `x` a `y` –, ktoré reprezentujú novú polohu cestujúceho v lietadle. V metóde aktualizujeme hodnotu dvojice `current_position`.
* `move(self)` – metóda reprezentuje pohyb cestujúceho v lietadle. Ak cestujúci ešte nebol pridaný do lietadla, funkcia vygeneruje `TypeError`. Ak sa cestujúci nachádza v lietadle, metóda vráti vždy dve hodnoty – novú pozíciu cestujúceho (rad sedadiel a číslovanie sedadiel resp. chodby) Pohyb cestujúceho vieme popísať niekoľkými pravidlami:
   1. kým cestujúci nie je v rade svojho miesta, ostane na chodbe a vždy urobí jeden krok dopredu ak nasledujúca pozícia je voľná
   2. ak cestujúci už stojí v rade svojho miesta, najprv uloží svoju batožinu (znížime hodnotu `self.bags` o 1, až kým nedosiahne 0, cestujúci ostane na svojej pozícii)
   3. ak cestujúci už stojí v rade svojho miesta a nemá batožinu, pozrie sa, či má voľnú cestu k svojmu miestu. Ak áno, tak si sadne, ak nie, poprosí ďalších cestujúcich, aby mu uvoľnili cestu (viď `move_row` a `return_row` v triede `Plane`).

In [ ]:
class Passenger:
    def __init__(self, row, seat, no_of_bags):
        if seat not in ['A', 'B', 'C', 'D', 'E', 'F']:
            raise ValueError("Seat must be a letter from A to F")

        self.row = row
        self.seat = seat
        self.bags = no_of_bags * 4

        self.plane = None
        self.current_position = [None, None]

    def __str__(self):
        return "<Passenger in seat: {}{}>".format(self.row, self.seat)

    def get_position(self):
        if self.plane is not None:
            return self.current_position[-1], self.current_position[0]
        else:
            return None

    def get_seat(self):
        x = SEAT_TO_IDX[self.seat]
        y = self.row
        return x, y

    def add_to_plane(self, plane):
        self.plane = plane
        self.current_position = [0, 3]

    def can_sit(self):
        if self.plane is None:
            return False

        if self.current_position[1] != 3:
            return False

        r = self.current_position[0]
        s_idx = SEAT_TO_IDX[self.seat]
        if self.seat in ['A', 'B', 'C']:
            for s in range(4, s_idx):
                if not self.plane.is_empty(r, s):
                    return False
        else:
            for s in range(s_idx + 1, 3):
                if not self.plane.is_empty(r, s):
                    return False

        return True

    def forced_to_move(self, x, y):
        self.current_position = [y, x]

    def move(self):
        if self.plane is None:
            raise TypeError("This passenger was not added to a plane yet")

        r, s = self.current_position
        if r < self.row and self.plane.is_empty(r + 1, s):
            self.current_position = [r + 1, s]
            return r + 1, s

        if r == self.row:
            if self.bags > 0:
                self.bags -= 1
                return r, s
            else:
                if self.can_sit():
                    self.current_position = [self.row, SEAT_TO_IDX[self.seat]]
                    self.plane.return_row(self.row)
                    return self.row, SEAT_TO_IDX[self.seat]
                else:
                    self.plane.move_row(self.row, self.seat)
                    return r, s

        return r, s

### 1.3. `Plane`

Trieda `Plane` definuje model lietadla a poskytuje prehľad o pozíciach jednotlivých pasažierov. Definuje nasledovné metódy a funkcionalitu:

* konštruktor triedy má jeden parameter: `length` (počet radov v lietadle). Trieda má nasledovné vnútorné premenné:
    * `self.length` – počet radov v lietadle (parameter `length`)
    * `self.seats` – dvojrozmerný zoznam zoznamov, kde každý riadok reprezentuje sedadlá s rovnakým písmenom a každý stĺpec reprezentuje jeden rad sedadiel. Na každej pozícii tohto poľa je zoznam cestujúcich, ktorí sa nachádzajú na danej pozícii. Pole bude mať dĺžku o 2 väčšiu ako je dĺžka lietadla aby sme mali 0. a n+1. rad (viď vizualizáciu lietadla vyššie).
* metódy `print_plane`, `print_seats` a `print_corridor` nám vykresľujú lietadlo tak, že na každej pozícii budeme mať počet cestujúcich na danej pozícii.
* `add_passengers(self, psg_list)` – metóda pridá cestujúcich zo zoznamu do lietadla. Má jeden parameter, `psg_list`, ktorý je zoznam s cestujúcimi. Pre každého cestujúceho zavoláme obdobnú metódu z triedy `Passenger`, ktorá ho pridá do lietadla a následne pridáme cestujúceho aj do zoznamu na pozícii `[3, 0]` (opačné poradie ako reprezentácia pozície v triede `Passenger`).
* `is_empty(self, row, seat)` – metóda vracia `True` ak zoznam na pozícii `[seat, row]` je prázdna, `False` v opačnom prípade. Ak daná pozícia neexistuje, vráti `True`.
* `move_row(self, row, seat_letter)` – metóda slúži na posunutie cestujúcich zo sedadiel s účelom uvoľnenia cesty pre ďalšieho cestujúceho k svojmu miestu. Metóda má dva parametre – rad a písmeno sedadla, do ktorého si chce cestujúci sadnúť. Metóda najprv zistí, či pozícia `[3, row + 1]` je voľná, a ak áno, posunie každého sediaceho cestujúceho na túto pozíciu (pridá ich do príslušného zoznamu). Ak pozícia nie je voľná, metóda nič nerobí, a nový cestujúci musí ďalej čakať.
* `return_row(self, row)` – metóda je opakom metódy `move_row`, t. j. vráti všetkých cestujúcich z pozície `[3, row  + 1]` na svoje miesta v jednom kroku, ak majú sedieť v danom rade.
* `move_passengers(self)` – metóda posunie všetkých cestujúcich na chodbe a aktualizuje ich pozíciu v dvojrozmernom poli. Pre zistenie novej pozície cestujúceho použijeme metódu `move` z triedy `Passenger`. Pre predídeniu deadlockov začneme s aktualizáciou pozície cestujúcich na konci lietadla (posledné miesto na chodbe).
* `boarding_finished(self)` – metóda zistí, či je nastupovanie dokončené. Vráti `True`, ak na chodbe už nie sú cestujúci a všetky sedadlá sú obsadené. V opačnom prípade vráti `False`.

In [ ]:
class Plane:
    def __init__(self, length):
        self.length = length
        self.seats = [[[] for i in range(self.length + 2)] for j in range(7)]

    def print_plane(self):
        for x in range(len(self.seats)):
            if x != 3:
                self.print_seats(x)
            else:
                self.print_corridor()

    def print_seats(self, seat_idx):
        row_str = "  | "
        for seat in self.seats[seat_idx][1:-1]:
            if len(seat) == 0:
                row_str += " O"
            else:
                row_str += " {}".format(len(seat))
        row_str += " |"
        print(row_str)

    def print_corridor(self):
        corridor_str = ""
        for position in self.seats[3]:
            if len(position) == 0:
                corridor_str += "  "
            else:
                corridor_str += " {}".format(len(position))
        print(corridor_str)

    def add_passengers(self, psg_list):
        for passenger in psg_list:
            passenger.add_to_plane(self)
            self.seats[3][0].append(passenger)

    def is_empty(self, row, seat):
        try:
            return not len(self.seats[seat][row])
        except IndexError:
            return True

    def move_row(self, row, seat_letter):
        if seat_letter in ["A", "B", "C"]:
            seat_list = [
                self.seats[4][row],
                self.seats[5][row],
                self.seats[6][row]
            ]
        else:
            seat_list = [
                self.seats[0][row],
                self.seats[1][row],
                self.seats[2][row]
            ]

        if self.is_empty(row + 1, 3):
            for psg_seat in seat_list:
                try:
                    psg = psg_seat[0]
                    psg_seat.remove(psg)
                    self.seats[3][row + 1].append(psg)
                    psg.forced_to_move(3, row + 1)
                except IndexError:
                    pass

    def return_row(self, row):
        cpy = self.seats[3][row + 1].copy()
        if not self.is_empty(row + 1, 3):
            for psg in cpy:
                y, x = psg.get_seat()
                p_y, p_x = psg.get_position()
                if x == row:
                    self.seats[3][row + 1].remove(psg)
                    self.seats[y][x].append(psg)
                    psg.forced_to_move(x, y)

    def move_passengers(self):
        for idx in range(len(self.seats[3]) - 1, -1, -1):
            for psg in self.seats[3][idx]:
                old_s, old_r = psg.get_position()
                new_r, new_s = psg.move()
                if old_s != new_s or old_r != new_r:
                    save_psg = psg
                    self.seats[3][idx].remove(psg)
                    self.seats[new_s][new_r].append(save_psg)
                else:
                    pass

    def boarding_finished(self):
        for pos in self.seats[3]:
            if len(pos) != 0:
                return False

        for row in self.seats[:3] + self.seats[4:]:
            for seat in row[1:-1]:
                if len(seat) != 1:
                    return False

        return True

## Krok 2: `Boarding`

V súbore `boarding.py` nájdeme všeobecnú triedu `Boarding` a jeho podtriedy, ktoré budú definovať konkrétne poradie nastupovania pasažierov. Trieda `Boarding` ale definuje všeobecnú funkcionalitu, a to konkrétne simuláciu nastupovania s cieľom zistiť počet krokov potrebných na usadenie všetkých cestujúcich (`run_simulation`), a vykonanie niekoľkých simulácií na zistenie priemernej dĺžky nastupovania (`test_boarding_method`).

V triede teda doplňte nasledovnú funkcionalitu:

* metóda `generate_boarding` ostane prázdna - bude špecifikovaná v podtriedach; metóda pripraví simuláciu tak, že nastaví lietadlo v členskej premennej `self.plane` (v ďalších metódach teda môžete pracovať s touto premennou).
* `run_simulation(self, plane_length)` – metóda, ktorá spustí simuláciu nastupovania cestujúcich do lietadla. Metóda má jeden parameter, a to dĺžku lietadla (počet radov), pre ktoré chceme vytvoriť simuláciu. V metóde máme vygenerovať poradie nastupovania cestujúcich (pomocou `generate_boarding`). Nastupovanie prebieha opätovným posúvaním cestujúcich až kým nastupovanie nie je dokončené – všetky miesta sú obsadené. Metóda vracia jednu hodnotu, počet potrebných krokov pre ukončenie nástupu. Metódu implementujeme iba v triede `Boarding` (podtriedy budú využívať túto implementáciu).
* `test_boarding_method(self, plane_length, no_simulation)` – metóda spustí niekoľko simulácií nastupovania danou metódou. Metóda má dva parametre: `plane_length` (počet radov v lietadle) a `no_simulation` (počet simulácií). Metóda vracia dve hodnoty: priemerný počet krokov potrebných na ukončenie nástupu, a zoznam výsledkov jednotlivých simulácií. Metódu implementujeme priamo v triede `Boarding` (podtriedy budú využívať túto implementáciu).

In [ ]:
class Boarding:
    def __init__(self):
        self.plane = None

    def generate_boarding(self, plane_length):
        # DO NOT IMPLEMENT HERE
        pass

    def run_simulation(self, plane_length):
        # TODO
        return None

    def test_boarding_method(self, plane_length, no_simulation):
        # TODO
        return None, None

## Krok 3: Definovanie rôznych metód nastupovania

V tomto kroku postupne implementujeme a otestujeme rôzne formy nastupovania pasažierov do lietadla. Nástup bude reprezentovaný vygenerovaním pasažierov podľa istého pravidla v metóde `generate_boarding`, ktorú prepíše každá podtrieda triedy `Boarding`. Vygenerovaných pasažierov následne pridáme do lietadla v danom poradí (zavoláme metódu `add_passengers` v `Plane`). Pri generovaní cestujúcich najprv vždy vytvoríme jednotlivé skupiny cestujúcich podľa ich miesta, následne náhodne premiešame túto skupinu cestujúcich a až potom ich pridáme do lietadla.

### Krok 3.1. Front-to-back

Najprv implementujeme metódu nástupu front-to-back, teda najprv nastúpia pasažieri, ktorí sedia vpredu. Cestujúcich rozdelíme na štyri skupiny na základe toho, v ktorom rade sedia (môžeme predpokladať, že lietadlo bude mať dĺžku - počet radov - deliteľnú 4). Na lietadlo najprv spustíme cestujúcich z najprednejšej  skupiny (v náhodnom poradí), potom cestujúcich z druhej najprednejšej  skupiny, atď. Ako poslední nastupujú cestujúci, ktorí sedia vzadu.

Implementujte metódu `generate_boarding` nasledovne:

1. vytvorte lietadlo a nastavte členskú premennú triedy `plane`
2. vypočítajte dĺžku jednej podskupiny (dĺžku lietadla delíme na štyri rovnaké časti)
3. pre každú podskupinu vygenerujte zoznam cestujúcich v skupine, následne ich poradie premiešajte v rámci skupiny (použite metódu [`shuffle`](docs.python.org/3/library/random.html)) a pridajte tento zoznam na koniec zoznamu všetkých pasažierov. Pasažieri nech majú náhodne vygenerovaný počet batožiny 1, 2 alebo 3 s váhami 0.3, 0.6 a 0.1 (použite metódu [`choices`](docs.python.org/3/library/random.html)).
4. pridajte zoznam všetkých cestujúcich do lietadla

In [ ]:
from random import choices, shuffle

class BoardingFTB(Boarding):
    def generate_boarding(self, plane_length):
        # TODO
        pass

### Krok 3.2. Testovanie

Otestujte vaše riešenie pomocou kódu nižšie. S takýmto nastavením parametrov by ste mali dostať priemerný počet krokov okolo 339.

In [ ]:
test = BoardingFTB()
avg, steps = test.test_boarding_method(8, 50)

print("Boarding took {} steps on average".format(avg))
print(steps)

### Krok 3.3. Back-to-front

Obdobným spôsobom vieme zadefinovať ďalší spôsob nástupu, a to back-to-front, kde pasažierov rozdelíme rovnako na štyri skupiny, ale najprv spustíme tých, ktorí sedia vzadu. Cestujúcich teda rozdelíme na štyri skupiny na základe toho, v ktorom rade sedia. Na lietadlo najprv spustíme cestujúcich z najzadnejšej skupiny (v náhodnom poradí), potom cestujúcich z druhej najzadnejšej skupiny, atď. Ako poslední nastupujú cestujúci, ktorí sedia vpredu. Môžeme predpokladať, že dĺžka lietadla, t.j. počet radov, bude deliteľná 4. Nástup otestujeme rovnakým spôsobom, v priemere potrebujeme okolo 295 krokov.

In [ ]:
class BoardingBTF(Boarding):
    def generate_boarding(self, plane_length):
        # TODO
        pass

In [ ]:
test = BoardingBTF()
avg, steps = test.test_boarding_method(8, 50)

print("Boarding took {} steps on average".format(avg))
print(steps)

### Krok 3.4. Window-to-aisle

V ďalsom spôsobe cestujúcich rozdelíme na tri skupiny na základe toho, kde sedia v rámci troch sedadiel (pri okne, v strede, alebo pri chodbe). Na lietadlo najprv spustíme cestujúcich sediacich pri okne, teda sedadlá A a F (v náhodnom poradí), potom cestujúcich sediacich v strede, teda sedadlá B a E, a na záver cestujúcich sediacich pri chodbe, teda sedadlá C a D. V priemere potrebujeme okolo 207 krokov.

In [ ]:
class BoardingWTA(Boarding):
    def generate_boarding(self, plane_length):
        # TODO
        pass

In [ ]:
test = BoardingWTA()
avg, steps = test.test_boarding_method(8, 50)

print("Boarding took {} steps on average".format(avg))
print(steps)

### Krok 3.5. Aisle-to-window

Cestujúcich rozdelíme na tri skupiny na základe toho, kde sedia v rámci troch sedadiel (pri okne, v strede, alebo pri chodbe). Na lietadlo najprv spustíme cestujúcich sediacich pri chodbe, teda sedadlá C a D (v náhodnom poradí), potom cestujúcich sediacich v strede, teda sedadlá B a E, a na záver cestujúcich sediacich pri okne, teda  sedadlá A a F. V priemere potrebujeme okolo 225 krokov.

In [ ]:
class BoardingATW(Boarding):
    def generate_boarding(self, plane_length):
        # TODO
        pass

In [ ]:
test = BoardingATW()
avg, steps = test.test_boarding_method(8, 50)

print("Boarding took {} steps on average".format(avg))
print(steps)

### Krok 3.6. Random

Cestujúcich nerozdelíme do žiadnych skupín, spustíme ich na lietadlo v náhodnom poradí. V priemere potrebujeme okolo 222 krokov.

In [ ]:
class BoardingRandom(Boarding):
    def generate_boarding(self, plane_length):
        # TODO
        pass

In [ ]:
test = BoardingRandom()
avg, steps = test.test_boarding_method(8, 50)

print("Boarding took {} steps on average".format(avg))
print(steps)

### Krok 3.7. Steffen's perfect

Steffenova metóda definuje optimálny nástup do lietadla. Cestujúcich spustíme na lietadlo podľa postupu definovaného Jasonom Steffenom: od okna po chodbu, zozadu každý druhý rad, striedavé strany. To znamená, že na lietadlo najprv spustíme cestujúcich na sedadlách v párnom rade a na sedadle A. Potom prídu cestujúci v párnom rade na sedadle F. V ďalšom kroku nastupujú cestujúci v nepárnych radoch na sedadle A. Nastupovanie cestujúcich sediacich pri okne ukončíme skupinou v nepárnych radoch na sedadle F. Nastupovanie pokračuje podľa rovnakého pravidla pre sedadlá B a E, resp. C a D:

![](sources/lab11/steffen.jpg)

V priemere potrebujeme okolo 152 krokov.

In [ ]:
class BoardingSteffen(Boarding):
    def generate_boarding(self, plane_length):
        # TODO
        pass

In [ ]:
test = BoardingSteffen()
avg, steps = test.test_boarding_method(8, 50)

print("Boarding took {} steps on average".format(avg))
print(steps)

## Doplňujúca úloha

Pre jednoduchšie porovnanie všetkých metód vygenerujte graf, ktorý bude vizualizovať distribúciu počtu krokov potrebných na ukončenie nástupu rôznymi metódami.